# Dashboard Component: Market Pulse

## Purpose
This query generates human-readable tags for technical indicators (Beta, Momentum, Volatility) to help users quickly assess the "mood" of a stock.

## Logic
*   **Beta Tag:** Categorizes stock volatility relative to the market (Aggressive vs Defensive).
*   **Momentum Tag:** Identifies short-term price trends (Skyrocketing vs Crashing).
*   **Risk Tag:** Highlights high-volatility conditions.

In [0]:
CREATE OR REPLACE VIEW view_dashboard_pulse AS
SELECT 
    symbol,
    valuation_timestamp,
    close_price,
    -- Handle NaN specifically for display
    round(cast(beta as double), 2) as beta,
    
    CASE 
        WHEN isnan(beta) OR beta IS NULL THEN '⚠️ Data Error'
        WHEN beta > 1.5 THEN '🔥 Wild Ride (Very High Risk)'
        WHEN beta > 1.1 THEN '🚀 Aggressive (Growth)'
        WHEN beta BETWEEN 0.9 AND 1.1 THEN '⚖️ Market Mover (Indexed)'
        WHEN beta < 0.9 THEN '🛡️ Defensive (Safe Haven)'
        ELSE 'Unknown'
    END as beta_tag,
    
    round(momentum * 100, 2) as momentum_pct,
    
    CASE 
        WHEN momentum > 0.05 THEN '🚀 Skyrocketing'
        WHEN momentum > 0 THEN '📈 Climbing'
        WHEN momentum > -0.05 THEN '📉 Cooling Down'
        ELSE '🩸 Crashing'
    END as trend_tag,
    
    round(volatility * 100, 2) as volatility_pct,
    
    CASE 
        WHEN volatility > 0.02 THEN 'High Anxiety (Volatile)'
        ELSE 'Calm Waters (Stable)'
    END as risk_tag

-- IMPORTANT: Make sure this FROM points to your new V10 Gold Table
FROM delta.`/Volumes/workspace/default/storage/serving/valuation_dashboard_v5`

WHERE 
    valuation_timestamp >= current_timestamp() - INTERVAL 24 HOURS
    -- THE FIX: Filter out NULLs and NaNs so they don't break the dashboard
    AND beta IS NOT NULL 
    AND NOT isnan(beta)
    AND volatility IS NOT NULL;